# Training Model mit Pointcloud manipulationen und dem Washington RGBD Datensatz

ToDo:


Umgebungsnoise !!!!




Version updated: 07.08.2019 (latest version)

In [ ]:
from pyntcloud import PyntCloud
import os
import numpy as np
import random
from random import randint, uniform
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras.callbacks import TensorBoard
from time import time
#from tensorflow.python.keras.utils import to_categorical

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Visualize the test data
import scipy.io as io
import scipy.ndimage as nd

# ---------------------------------------------- Variables -------------------------------------------------------------

DATADIR = "data/washington_rgbd_dataset/"
CATEGORIES = ["apple", "coffee_mug", "greens", "onion", "soda_can", "ball", "comb", "hand_towel", "orange", "sponge",
              "banana", "dry_battery", "instant_noodles", "peach", "stapler", "bell_pepper", "flashlight", "keyboard",
              "pear", "tomato", "binder", "food_bag", "kleenex", "pitcher", "toothbrush", "bowl", "food_box", "lemon",
              "plate", "toothpaste", "calculator", "food_can", "lightbulb", "pliers", "water_bottle", "camera", 
              "food_cup", "lime", "potato", "cap", "food_jar", "marker", "rubber_eraser", "cell_phone", "garlic",
              "mushroom", "scissors", "cereal_box", "glue_stick", "notebook", "shampoo"] # 51 Cats
r = 0 

# max(X,Y,Z) of Pointcloud
pX_max = 0
pY_max = 0
pZ_max = 0

# Scene Container creating the Batches for Traning
scene_container_x = {}
scene_container_y = {}

# dict for evaluating past losses
loss_dict = []
# Counter on how many scenes the Network has been trained
counter_trainingobjects = 0

# Number of Scenes for Batch-Training
number_scenes = 32

# ---------------------------------------------- Functions/Classes ----------------------------------------------------


class LossHistory(tf.keras.callbacks.Callback):

    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))


def switch_pos(argument):
    """Switcher for positional change of PCD in the scene"""
    pos1 = uniform(0,1)
    pos2 = uniform(1.1,2)
    pos3 = uniform(2.1,3)
    pos4 = uniform(3.1,4)
    pos5 = uniform(4.1,5)
    pos6 = uniform(5.1,6)
    pos7 = uniform(6.1,7)
    pos8 = uniform(7.1,8)

    switcher = {
        0: (pos1, pos6, pos7),
        1: (pos2, pos4, pos6),
        2: (pos3, pos2, pos5),
        3: (pos4, pos7, pos1),
        4: (pos5, pos3, pos2),
        5: (pos6, pos5, pos6),
        6: (pos8, pos1, pos8)
    }

    return switcher.get(argument, "nothing")

def pc_normalize(pointcloud):
    
    l = pointcloud.shape[0]
    centroid = np.mean(pointcloud, axis=0)
    pointcloud = pointcloud - centroid
    m = np.max(np.sqrt(np.sum(pointcloud**2, axis=1)))
    pointcloud = pointcloud / m
    return pointcloud


# --------------------------------------------- Network Setup ---------------------------------------------------------


# To activate type in console "tensorboard --logdir=logs/" and then start training and open link from consoleinput
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))


model = tf.keras.models.Sequential([
    tf.keras.layers.Reshape((128, 128, 128, 1), input_shape=(128, 128, 128)),  # Add 1 dim for 3D CNN
    tf.keras.layers.Conv3D(filters=32, kernel_size=(3,3,3), strides=2, activation='relu'),
    tf.keras.layers.Conv3D(filters=64, kernel_size=(3,3,3), strides=2, activation='relu'),
    tf.keras.layers.Conv3D(filters=128, kernel_size=(3,3,3), strides=2, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])

In [ ]:
# -------------------------------------Begin Create Pointcloudscene --------- ------------------------------------------

# Main-Loop for Training till User interruption
while True: 
    
    try: 
        # Loop iteration
        v = 0  # Till number of Scene in Scene Container

        while v < number_scenes: # Adds Scenes to the Scene Container

            # Creates an empty Pointcloud Scene
            pointcloudscene = np.empty((0, 3), dtype=float)

            # How many Objects per Scene
            number_objects = randint(2, 7)
            #print(number_objects)

            # Loop iterations
            i = 0  # Number of pointcloud per Scene
            y = 0  # Randomn Object in datapath

            while i < number_objects:  # Number of Pointcloud per scene

                working_category = CATEGORIES[randint(0, 50)] # What Category is selected

                path = os.path.join(DATADIR, working_category) # for 51 Categories
                #number_folder = len(next(os.walk(path))[1]) # Number of Folders in Category            

                #path = path + "/" + working_category + "_" + str(randint(1, number_folder)) # Add random folder to path

                path = path + "/" + random.choice(os.listdir(path))

                #files = next(os.walk(path))[2]
                #number_files = len(files) # Number of pcd files in randomly selected Category
                #print("nb files", number_files)

                path = path + "/" + random.choice(os.listdir(path)) # Picks Randomn File from path
                #print(path)

                # Load ModelNet10 Data to PyntCloud Object
                pointcloud = PyntCloud.from_file(path)



                #df_pointcloud = pd.DataFrame(pointcloud.points)
                #del df_pointcloud['imX']
                #del df_pointcloud['imY']
                #del df_pointcloud['red']
                #del df_pointcloud['green']
                #del df_pointcloud['blue']
                #print(df_pointcloud)


                #pointcloud = PyntCloud(df_pointcloud)
                #pointcloud = pointcloud.get_sample("mesh_random", n=10000, rgb=False, normals=False, as_PyntCloud=False) 




                # ------------------------------- Pointcloud manipulations ------------------------------------------------#

                #Pointcloud Normaliazation

                pointcloud = pointcloud.xyz
                normalized_pointcloud = pc_normalize(pointcloud)

                p_x = normalized_pointcloud[:, 0]
                p_y = normalized_pointcloud[:, 1]
                p_z = normalized_pointcloud[:, 2]

                normalized_pointcloud = pd.DataFrame(zip(p_x, p_y, p_z), columns=['x', 'y', 'z'])
                normalized_pointcloud = PyntCloud(normalized_pointcloud)

                # Add Noise to each Pointcloud
                noise = pd.DataFrame(np.random.uniform(0.0,0.2,[len(normalized_pointcloud.points.x),3]),columns=['x','y','z'])
                noisy_pointcloud = normalized_pointcloud.points.add(noise)
                noisy_pointcloud = PyntCloud(noisy_pointcloud)

                # Change Position of Pointcloud 
                pos = switch_pos(i)
                final_pointcloud = noisy_pointcloud.xyz + ([pos])

                # Append Pointcloud to Scene
                pointcloudscene = np.append(pointcloudscene, final_pointcloud, axis=0)

                # Create Labeled Pointcloudscene for allocation Pointclouds with clusters
                #labeled_pointcloudscene.update({i: npy_cloud})

                i = i + 1

            # Create pandas dateframe to convert scene back to pyntcloud Object
            p_x = pointcloudscene[:, 0]
            p_y = pointcloudscene[:, 1]
            p_z = pointcloudscene[:, 2]

            df_pointcloudscene = pd.DataFrame(zip(p_x, p_y, p_z), columns=['x', 'y', 'z'])
            pyntcloud_scene = PyntCloud(df_pointcloudscene)


            # ------------------------------------------ Voxilisation -----------------------------------------------------

            # create Voxelgrid from Pointcloudscene
            voxelgrid_id = pyntcloud_scene.add_structure("voxelgrid", n_x=128, n_y=128, n_z=128)
            voxelscene = pyntcloud_scene.structures[voxelgrid_id]

            # Create binary array from Voxelscene 
            binary_voxelscene = voxelscene.get_feature_vector(mode="binary")

            # Create a Voxelscene Container of Scenes
            scene_container_x[v]=binary_voxelscene
            scene_container_y[v]=number_objects

            v = v + 1

        v = 0

        #---------------------------------------- Prepare data for Network input ------------------------------------------

        # X = datainput for Network
        X = np.zeros((number_scenes,128,128,128), dtype=float) 
        k = 0

        for key, value in scene_container_x.items():
            temp = [value]
            X[k] = np.concatenate(temp)
            k = k + 1

        # Y = label input for Network
        Y = []
        for key, value in scene_container_y.items():
            temp = [value]
            Y.append(temp)
        Y = np.asarray(Y)
        Y = tf.keras.utils.to_categorical(Y, 10)

        # --------------------- Training the Network by incrementally call fit function ------------------------------------

        history = LossHistory()
        model.fit(X, Y, batch_size=8, epochs=1, verbose=1, callbacks=[tensorboard, history])
        counter_trainingobjects = counter_trainingobjects + 1

        if (history.losses[0] < 0.001):
            model.save('savedmodel/k_model_CNN.h5')
            print("Model has been saved and trained on", counter_trainingobjects*32, "Voxelscenes")
   
    except: continue

